In [1]:
import os
import re
import glob
import pandas as pd

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/04_prophage_rumen_mags/")

In [3]:
dramv_res = glob.glob('dram_annotation/**/annotations.tsv', recursive = True)

In [4]:
dramv_df = []
for file in dramv_res:
    df = pd.read_csv(file, sep = '\t')
    dramv_df.append(df)

In [5]:
vcontigs_for_manual_curation = pd.concat(dramv_df)

In [6]:
amg_containing = list(set(vcontigs_for_manual_curation.query('amg_flags == amg_flags')[vcontigs_for_manual_curation.query('amg_flags == amg_flags').amg_flags.str.contains("M")].query('auxiliary_score < 4').fasta))

In [7]:
vcontigs_for_manual_curation.rename(columns = {'Unnamed: 0': 'gene_name'}, inplace = True)
vcontigs_for_manual_curation.drop("scaffold", axis = 1, inplace = True)

In [8]:
with open("prophage_rumen_mags_category2.txt", "r") as file:
    category2 = file.read().split('\n')

In [9]:
with open("amr/amr_containing_contigs.txt", 'r') as file:
    amr_contigs = file.read().split("\n")

In [10]:
with open("non-cryptic-prophage.txt", 'r') as file:
    active_prophage = file.read().split("\n")

In [11]:
prophage_hgt_for_manual_curation = vcontigs_for_manual_curation.query('fasta in @amg_containing').query('fasta in @category2').query('fasta not in @active_prophage')
prophage_amr_for_manual_curation = vcontigs_for_manual_curation.query('fasta in @amr_contigs')
prophage_amg_for_manual_curation = vcontigs_for_manual_curation.query('fasta in @amg_containing').query('fasta in @active_prophage')

In [12]:
prophage_hgt_for_manual_curation.loc[:, 'hits'] = prophage_hgt_for_manual_curation['kegg_hit'] + prophage_hgt_for_manual_curation['cazy_hits']  + prophage_hgt_for_manual_curation['viral_hit']  + prophage_hgt_for_manual_curation['pfam_hits'] + prophage_hgt_for_manual_curation['vogdb_hit']
prophage_amr_for_manual_curation.loc[:, 'hits'] = prophage_amr_for_manual_curation['kegg_hit'] + prophage_amr_for_manual_curation['cazy_hits']  + prophage_amr_for_manual_curation['viral_hit']  + prophage_amr_for_manual_curation['pfam_hits'] + prophage_amr_for_manual_curation['vogdb_hit']
prophage_amg_for_manual_curation.loc[:, 'hits'] = prophage_amg_for_manual_curation['kegg_hit'] + prophage_amg_for_manual_curation['cazy_hits']  + prophage_amg_for_manual_curation['viral_hit']  + prophage_amg_for_manual_curation['pfam_hits'] + prophage_amg_for_manual_curation['vogdb_hit']

/tmp/slurmtmp.15368821/ipykernel_70899/1728240214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prophage_amr_for_manual_curation.loc[:, 'hits'] = prophage_amr_for_manual_curation['kegg_hit'] + prophage_amr_for_manual_curation['cazy_hits']  + prophage_amr_for_manual_curation['viral_hit']  + prophage_amr_for_manual_curation['pfam_hits'] + prophage_amr_for_manual_curation['vogdb_hit']


In [13]:
lps_filtering_hgt = prophage_hgt_for_manual_curation[["fasta", "gene_position",  "end_position", "hits"]].query('hits == hits')
lps_filtering_amr = prophage_amr_for_manual_curation[["fasta", "gene_position",  "end_position", "hits"]].query('hits == hits')
lps_filtering_amg = prophage_amg_for_manual_curation[["fasta", "gene_position",  "end_position", "hits"]].query('hits == hits')

In [14]:
lps_keywords = ['carbohydrate kinase', 'carbohydrate-kinase', 'glycosyltransferase family 11 ', 'glycosyltransferase family 25', 'glycosyl transferase family 11 ', 'glycosyl transferase family 25', 'Glycosyltransferase family 11 ', 'Glycosyltransferase family 25', 'GT11 ', 'GT25', 'nucleotide sugar epimerase', 'nucleotide sugar-epimerase', 'nucleotide-sugar epimerase', 'nucleotide-sugar-epimerase', 'nucleotidyltransferase', 'nucleotidyl transferase', 'nucleotidyl-transferase', 'plasmid stability', 'endonuclease']

In [15]:
def filter_lps(df):
    furthur_check = []
    for index, row in df.iterrows():
        hits = row['hits']
        votu = row['fasta']
        if votu in furthur_check:
            continue
        else:
            for lps in lps_keywords:

                if lps.lower() in hits.lower():
                    furthur_check.append(votu)
                else:
                    continue
    
    return furthur_check
                    

In [16]:
filter_lps(lps_filtering_hgt)

[]

In [17]:
filter_lps(lps_filtering_amr)

[]

In [18]:
filter_lps(lps_filtering_amg)

[]

# No LPS-containing viral contigs

In [19]:
prophage_amg_for_manual_curation_reformat = prophage_amg_for_manual_curation.get(['gene_name', 'fasta', 'kegg_hit', 'viral_hit', 'pfam_hits'])
prophage_amr_for_manual_curation_reformat = prophage_amr_for_manual_curation.get(['gene_name', 'fasta', 'kegg_hit', 'viral_hit', 'pfam_hits'])
prophage_hgt_for_manual_curation_reformat = prophage_hgt_for_manual_curation.get(['gene_name', 'fasta', 'kegg_hit', 'viral_hit', 'pfam_hits'])

In [20]:
group_amg = prophage_amg_for_manual_curation_reformat.groupby('fasta').groups.keys()
group_amr = prophage_amr_for_manual_curation_reformat.groupby('fasta').groups.keys()
group_hgt = prophage_hgt_for_manual_curation_reformat.groupby('fasta').groups.keys()


In [21]:
import re
def find_text(ser, string):
    res = ser[ser.str.contains(string, flags=re.IGNORECASE)]
    if res.empty:
        res = ''
    else:
        res = str(list(res))
    return res
def get_hallmark(df):
    kegg_hit = df['kegg_hit'].fillna('')
    viral_hit = df['viral_hit'].fillna('')
    pfam_hit =  df['pfam_hits'].fillna('')
    
       
    kegg_capsid = find_text(kegg_hit, 'capsid')
    kegg_portal = find_text(kegg_hit, 'portal')
    kegg_coat = find_text(kegg_hit, 'portal')
    kegg_baseplate = find_text(kegg_hit, 'baseplate')
    kegg_head = find_text(kegg_hit, 'head')
    kegg_tail = find_text(kegg_hit, 'tail')
    kegg_tail_fiber = find_text(kegg_hit, 'tail fiber')
    kegg_terminase = find_text(kegg_hit, 'terminase')
    kegg_integrase = find_text(kegg_hit, 'integrase')
    kegg_transposase = find_text(kegg_hit, 'transposase')
    
    viral_capsid = find_text(viral_hit, 'capsid')
    viral_portal = find_text(viral_hit, 'portal')
    viral_coat = find_text(viral_hit, 'portal')
    viral_baseplate = find_text(viral_hit, 'baseplate')
    viral_head = find_text(viral_hit, 'head')
    viral_tail = find_text(viral_hit, 'tail')
    viral_tail_fiber = find_text(viral_hit, 'tail fiber')
    viral_terminase = find_text(viral_hit, 'terminase')
    viral_integrase = find_text(viral_hit, 'integrase')
    viral_transposase = find_text(viral_hit, 'transposase')
    
    pfam_capsid = find_text(pfam_hit, 'capsid')
    pfam_portal = find_text(pfam_hit, 'portal')
    pfam_coat = find_text(pfam_hit, 'portal')
    pfam_baseplate = find_text(pfam_hit, 'baseplate')
    pfam_head = find_text(pfam_hit, 'head')
    pfam_tail = find_text(pfam_hit, 'tail')
    pfam_tail_fiber = find_text(pfam_hit, 'tail fiber')
    pfam_terminase = find_text(pfam_hit, 'terminase')
    pfam_integrase = find_text(pfam_hit, 'integrase')
    pfam_transposase = find_text(pfam_hit, 'transposase')
    
    all_structure = kegg_capsid + kegg_portal + kegg_coat + kegg_baseplate + kegg_head + kegg_tail + kegg_tail_fiber + kegg_terminase + kegg_integrase + kegg_transposase + viral_capsid + viral_portal + viral_coat + viral_baseplate + viral_head + viral_tail + viral_tail_fiber + viral_terminase + viral_integrase + viral_transposase + pfam_capsid + pfam_portal + pfam_coat + pfam_baseplate + pfam_head + pfam_tail + pfam_tail_fiber + pfam_terminase + pfam_integrase + pfam_transposase
    structure_no_tail_no_integrase = kegg_capsid + kegg_portal + kegg_coat + kegg_baseplate + kegg_head + kegg_terminase + viral_capsid + viral_portal + viral_coat + viral_baseplate + viral_head + viral_terminase + pfam_capsid + pfam_portal + pfam_coat + pfam_baseplate + pfam_head + pfam_terminase 
    integrase_or_transposase =  kegg_integrase + kegg_transposase + viral_integrase + viral_transposase + pfam_integrase + pfam_transposase
        
    return all_structure, structure_no_tail_no_integrase, integrase_or_transposase, kegg_capsid, kegg_portal, kegg_coat, kegg_baseplate, kegg_head, kegg_tail, kegg_tail_fiber, kegg_terminase, kegg_integrase, kegg_transposase, viral_capsid, viral_portal, viral_coat, viral_baseplate, viral_head, viral_tail, viral_tail_fiber, viral_terminase, viral_integrase, viral_transposase, pfam_capsid, pfam_portal, pfam_coat, pfam_baseplate, pfam_head, pfam_tail, pfam_tail_fiber, pfam_terminase, pfam_integrase, pfam_transposase
        
    
    

In [22]:
dramv_amg_for_manual_curation_final = pd.DataFrame(list(prophage_amg_for_manual_curation_reformat.groupby('fasta').apply(get_hallmark)), index = group_amg, columns = ["all_structure", "structure_no_tail_no_integrase", "integrase_or_transposase", 'kegg_capsid', 'kegg_portal', 'kegg_coat', 'kegg_baseplate', 'kegg_head', 'kegg_tail', 'kegg_tail_fiber', 'kegg_terminase', 'kegg_integrase', 'kegg_transposase', 'viral_capsid', 'viral_portal', 'viral_coat', 'viral_baseplate', 'viral_head', 'viral_tail', 'viral_tail_fiber', 'viral_terminase', 'viral_integrase', 'viral_transposase', 'pfam_capsid', 'pfam_portal', 'pfam_coat', 'pfam_baseplate', 'pfam_head', 'pfam_tail', 'pfam_tail_fiber', 'pfam_terminase', 'pfam_integrase', 'pfam_transposase'])
dramv_amr_for_manual_curation_final = pd.DataFrame(list(prophage_amr_for_manual_curation_reformat.groupby('fasta').apply(get_hallmark)), index = group_amr, columns = ["all_structure", "structure_no_tail_no_integrase", "integrase_or_transposase", 'kegg_capsid', 'kegg_portal', 'kegg_coat', 'kegg_baseplate', 'kegg_head', 'kegg_tail', 'kegg_tail_fiber', 'kegg_terminase', 'kegg_integrase', 'kegg_transposase', 'viral_capsid', 'viral_portal', 'viral_coat', 'viral_baseplate', 'viral_head', 'viral_tail', 'viral_tail_fiber', 'viral_terminase', 'viral_integrase', 'viral_transposase', 'pfam_capsid', 'pfam_portal', 'pfam_coat', 'pfam_baseplate', 'pfam_head', 'pfam_tail', 'pfam_tail_fiber', 'pfam_terminase', 'pfam_integrase', 'pfam_transposase'])
dramv_hgt_for_manual_curation_final = pd.DataFrame(list(prophage_hgt_for_manual_curation_reformat.groupby('fasta').apply(get_hallmark)), index = group_hgt, columns = ["all_structure", "structure_no_tail_no_integrase", "integrase_or_transposase", 'kegg_capsid', 'kegg_portal', 'kegg_coat', 'kegg_baseplate', 'kegg_head', 'kegg_tail', 'kegg_tail_fiber', 'kegg_terminase', 'kegg_integrase', 'kegg_transposase', 'viral_capsid', 'viral_portal', 'viral_coat', 'viral_baseplate', 'viral_head', 'viral_tail', 'viral_tail_fiber', 'viral_terminase', 'viral_integrase', 'viral_transposase', 'pfam_capsid', 'pfam_portal', 'pfam_coat', 'pfam_baseplate', 'pfam_head', 'pfam_tail', 'pfam_tail_fiber', 'pfam_terminase', 'pfam_integrase', 'pfam_transposase'])

In [23]:
checkv_res = pd.read_csv("prophage_rumen_mags_vs2_vibrant_intersection_checkv/quality_summary.tsv", sep = '\t')

In [24]:
checkv_res_short = checkv_res[["contig_id", "contig_length", "gene_count", "viral_genes", "host_genes"]]
dramv_amg_for_manual_curation_final = pd.merge(checkv_res_short, dramv_amg_for_manual_curation_final, left_on = "contig_id", right_index = True, how = "inner")
dramv_amr_for_manual_curation_final = pd.merge(checkv_res_short, dramv_amr_for_manual_curation_final, left_on = "contig_id", right_index = True, how = "inner")
dramv_hgt_for_manual_curation_final= pd.merge(checkv_res_short, dramv_hgt_for_manual_curation_final, left_on = "contig_id", right_index = True, how = "inner")

In [25]:
dramv_hgt_for_manual_curation_final.reset_index().rename(columns= {"index":"vcontigs"}).to_csv("prophage_hgt_check.tsv", sep="\t", index = None)
dramv_amg_for_manual_curation_final.reset_index().rename(columns= {"index":"vcontigs"}).to_csv("prophage_amg_check.tsv", sep="\t", index = None)
dramv_amr_for_manual_curation_final.reset_index().rename(columns= {"index":"vcontigs"}).to_csv("prophage_amr_check.tsv", sep="\t", index = None)

In [26]:
prophage_hgt_for_manual_curation.to_csv("prophage_hgt_for_manual_curation.tsv", sep="\t", index = None)
prophage_amr_for_manual_curation.to_csv("prophage_amr_for_manual_curation.tsv", sep="\t", index = None)
prophage_amg_for_manual_curation.to_csv("prophage_amg_for_manual_curation.tsv", sep="\t", index = None)